In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# default_exp traindl

# 用户的点击当做一个序列

# lib导入

In [150]:
# export
import gc
import tqdm
import os
from tx.config import * 
from tx.eda import * 
from loguru import logger
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行
# from multiprocessing.dummy import Pool
from sklearn.metrics import classification_report, accuracy_score
import lightgbm as lgb
import matplotlib.pyplot as plt

%matplotlib inline

In [10]:
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))  